In [ ]:
# Incase Of Update
response = requests.get('https://db.ygoprodeck.com/api/v7/cardinfo.php')
json_response = response.json()
dataset = pd.DataFrame(json_response['data'])

dataset.to_csv('Dataset/Yugioh_Database.csv')

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import warnings

import random as random
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


warnings.simplefilter(action='ignore', category=FutureWarning)
plt.style.use('Solarize_Light2')
pd.set_option('display.max_columns', 20)

#import requests
#import itertools

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession
# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)


In [2]:
# ========================================================================
'''Load Dataset'''
dataset = pd.read_csv('Dataset/Yugioh_Database.csv')
dataset = dataset.drop(['Unnamed: 0' , 'frameType' , 'archetype' , 'ygoprodeck_url' , 'card_sets' , 'card_images' , 'card_prices' , 'banlist_info'],axis=1)
dataset = dataset[dataset['type'] != 'XYZ Pendulum Effect Monster']
dataset = dataset[dataset['type'] != 'Synchro Pendulum Effect Monster']
dataset = dataset[dataset['type'] != 'Fusion Pendulum Effect Monster']
dataset = dataset[dataset['type'] != 'Pendulum Effect Monster']
dataset = dataset[dataset['type'] != 'Pendulum Normal Monster']

dataset = dataset[dataset['type'] != 'Skill Card']
dataset = dataset[dataset['type'] != 'Monster Token']
# Staple Removal
dataset = dataset[[not i for i in dataset['name'].isin(['Ash Blossom & Joyous Spring' , 'Effect Veiler' , 'Ghost Ogre & Snow Rabbit' ,'Ghost Belle & Haunted Mansion',
                                                        'Infinite Impermanence' , 'Red Reboot' , 'Called by the Grave' , 'Forbidden Droplet' , 'Crossout Designator',
                                                        'Nibiru, the Primal Being', 'Harpie\"s Feather Duster' , 'Lightning Storm' , 'Pot of Prosperity' , 'Pot of Desires',
                                                        'Pot of Duality' , 'Pot of Extravagance' , 'Triple Tactics Talents' , 'Torrential Tribute' , 'Dark Ruler No More' , 
                                                        'Red Reboot', 'D.D. Crow' , 'PSY-Framegear Gamma' , 'Maxx \"C\"' , 'Dimension Shifter' , 'Droll & Lock Bird' , 
                                                        'Accesscode Talker', 'Apollousa, Bow of the Goddess', 'Borreload Dragon' , 'Borrelsword Dragon', 'Knightmare Unicorn',
                                                        'Predaplant Verte Anaconda' , 'Knightmare Phoenix' , 'Knightmare Cerberus' , 'Underworld Goddess of the Closed World',
                                                        'Borreload Savage Dragon'])]]

dataset.loc[dataset['type']=='Normal Monster', ['desc']] = 'NoInfo'
dataset = dataset.fillna('0')
dataset['level'] = dataset['level'].astype('int32')



# ========================================================================
'''Create Tokenized sequence database'''


df = dataset['desc']         #Tokenizer is only trained on desc and based on that . Otherwise if trained on names it would blow vocab up to absurd amounts
Sliced_df = dataset[['level' , 'race' , 'type' , 'attribute' , 'name' , 'desc']]

for i in range(1,11,2):
    Sliced_df.insert(loc=i, column='A'+str(i), value=-1)        # Adds seperator columns


Sliced_df = Sliced_df.reset_index(drop=True)            # Need to reset the indexes so they are consistent
df = df.reset_index(drop=True)                              

tokenizer = Tokenizer(filters='\r , \n , \" ') # Speech marks stop names from being recognised by tokenizer
tokenizer.fit_on_texts(df)
tokenizer.word_index['0'] = 0           #Signifies Empty values
tokenizer.word_index['-1'] = -1           #Signifies Seperators

sequences = []
padded_sequences = []
Tokenized_sequence_database = []
count = 0
for i in Sliced_df.astype('string').to_numpy():
    
    sequences.append(tokenizer.texts_to_sequences(i))
    

for i in range(0,11):
    padded_sequences.append( pad_sequences(np.array(sequences , dtype='object')[:,i], padding='post') ) 

Tokenized_sequence_database = np.concatenate(([padded_sequences[i] for i in range(11)]) , axis=1 )

    




In [419]:
Sliced_df

,level,A1,race,A3,type,A5,attribute,A7,name,A9,desc
0,0,-1,Continuous,-1,Spell Card,-1,0,-1,"""A"" Cell Breeding Device",-1,"During each of your Standby Phases, put 1 A-Co..."
1,0,-1,Continuous,-1,Spell Card,-1,0,-1,"""A"" Cell Incubator",-1,Each time an A-Counter(s) is removed from play...
2,0,-1,Quick-Play,-1,Spell Card,-1,0,-1,"""A"" Cell Recombination Device",-1,Target 1 face-up monster on the field; send 1 ...
3,0,-1,Quick-Play,-1,Spell Card,-1,0,-1,"""A"" Cell Scatter Burst",-1,"Select 1 face-up ""Alien"" monster you control. ..."
4,0,-1,Equip,-1,Spell Card,-1,0,-1,"""Infernoble Arms - Almace""",-1,While this card is equipped to a monster: You ...
...,...,...,...,...,...,...,...,...,...,...,...
12456,4,-1,Beast,-1,Effect Monster,-1,LIGHT,-1,ZW - Sleipnir Mail,-1,"You can target 1 ""Utopia"" monster you control;..."
12457,4,-1,Beast,-1,Effect Monster,-1,LIGHT,-1,ZW - Sylphid Wing,-1,"You can only control 1 ""ZW - Sylphid Wing"". Yo..."
12458,5,-1,Dragon,-1,Effect Monster,-1,WIND,-1,ZW - Tornado Bringer,-1,"You can target 1 ""Utopia"" monster you control;..."
12459,4,-1,Aqua,-1,Effect Monster,-1,EARTH,-1,ZW - Ultimate Shield,-1,When this card is Normal or Special Summoned: ...


In [21]:
def Deck_Loader(directory):
    '''Loads Decks from Deck_Lists.txt as arrays and stores those arrays in altered'''
    file = open(directory , 'r')
    read = file.readlines()
    Deck_Array = []
    flag = False

    temp=[]

    for count,line in enumerate(read):
        
        if '//' in read[count]:
            flag = not flag
        
        if flag:
            
            read[count] = read[count].replace('\n','')
            
            if ('=='  in read[count]) or ('//'  in read[count])  :
                pass
                
            else:
                for i in range(int(read[count][0])):
                    temp.append(read[count][1:].strip())          #skip appending also remove white space

        if (not flag) or (count == len(read) - 1):
            Deck_Array.append(temp)
            temp = []
            flag = not flag
            
            
    file.close()
    return Deck_Array 


def stitcher(Deck_Index , Deck_Array , Random_Flag , Input_Array):
    '''Picks 5 random cards from a certain deck in a deck array and stitches them together. Or just stitches cards from input array together.'''
    if Random_Flag:
        decider = [random.choice(Deck_Array[Deck_Index]) for _ in range(5)]
    else:
        decider = Input_Array

    ###If decider picks all staples then error can occur with an empty array hence recalls stitcher function.    
    try:
        output = np.concatenate(([Tokenized_sequence_database[i] for i in Sliced_df[Sliced_df['name'].isin(decider)].index.values]) )
    except:
        print(decider)
        return stitcher(Deck_Index , Deck_Array , Random_Flag , Input_Array)

    if len(output) != 905:
        return stitcher(Deck_Index , Deck_Array , Random_Flag , Input_Array)
    else:
        return output

def no_match_generator(length , Deck_Array):
    '''Generated Datapoints which correspond to a card that doesnt relate to a given group of decider cards'''
    out = []
    for _ in range(length):
        ran_deck = random.choice(Deck_Array)
        ran_indx = random.choice(Sliced_df[Sliced_df['name'].isin(ran_deck)].index.values)

        temp = []
        #Incase card chosen is a staple, it skips it.
        try:
            subject_card = np.concatenate( ( Tokenized_sequence_database[ran_indx]  , np.zeros(724)) , axis=None )
            decider_cards = np.concatenate([random.choice(Tokenized_sequence_database) for _ in range(5)])

            temp.append(decider_cards)
            temp.append(subject_card)
            temp.append([0])

            out.append(temp)
        except:
            pass
    
    return out

def Dataset_Builder(directory , include_no_match):
    '''Builds a dataset with some specificaiton. This could be a training/validation dataset or a experimentation dataset'''
    altered = Deck_Loader(directory)
    Built_Dataset = []
    for deck_index,deck in enumerate(altered):
        
        for card_index in Sliced_df[Sliced_df['name'].isin(deck)].index.values:
            
            subject_card =  np.concatenate( (Tokenized_sequence_database[card_index] , np.zeros(724)) , axis=None ) # Extends subject to equal length of decider cards
            for _ in range(2):
                temp = []
                decider_cards = stitcher(deck_index , altered , True, [])
                temp.append(decider_cards)
                temp.append(subject_card)
                temp.append([1])
                Built_Dataset.append(temp)


    if (include_no_match):
        Built_Dataset.extend( no_match_generator(len(Built_Dataset) , altered) )
    random.shuffle(Built_Dataset)
    random.shuffle(Built_Dataset)
    random.shuffle(Built_Dataset)
    return Built_Dataset



Training_Validation_Dataset = Dataset_Builder('Dataset/Deck_Lists.txt' , True)
pd.DataFrame(Training_Validation_Dataset)


['Knightmare Phoenix', 'Called by the Grave', 'Infinite Impermanence', 'Infinite Impermanence', 'Infinite Impermanence']
['Infinite Impermanence', 'Ash Blossom & Joyous Spring', 'Knightmare Phoenix', 'Knightmare Cerberus', 'Pot of Prosperity']


,0,1,2
0,"[7, -1, 246, 0, -1, 170, 8, 0, 0, -1, 279, -1,...","[0.0, -1.0, 766.0, 0.0, -1.0, 120.0, 8.0, 0.0,...",[0]
1,"[97, -1, 866, 0, -1, 21, 8, 0, 0, -1, 297, -1,...","[276.0, -1.0, 286.0, 146.0, -1.0, 21.0, 8.0, 0...",[0]
2,"[0, -1, 711, 0, -1, 80, 5, 0, 0, -1, 0, -1, 45...","[0.0, -1.0, 374.0, 0.0, -1.0, 120.0, 8.0, 0.0,...",[1]
3,"[194, -1, 374, 0, -1, 75, 8, 0, 0, -1, 198, -1...","[0.0, -1.0, 411.0, 0.0, -1.0, 120.0, 8.0, 0.0,...",[0]
4,"[0, -1, 41, 0, -1, 80, 5, 0, 0, -1, 0, -1, 636...","[663.0, -1.0, 766.0, 0.0, -1.0, 21.0, 8.0, 0.0...",[0]
...,...,...,...
1583,"[0, -1, 59, 0, -1, 80, 5, 0, 0, -1, 0, -1, 172...","[276.0, -1.0, 286.0, 146.0, -1.0, 21.0, 8.0, 0...",[1]
1584,"[7, -1, 286, 146, -1, 21, 8, 0, 0, -1, 297, -1...","[194.0, -1.0, 363.0, 0.0, -1.0, 85.0, 8.0, 0.0...",[1]
1585,"[0, -1, 711, 0, -1, 80, 5, 0, 0, -1, 0, -1, 31...","[0.0, -1.0, 327.0, 0.0, -1.0, 80.0, 5.0, 0.0, ...",[1]
1586,"[0, -1, 59, 0, -1, 80, 5, 0, 0, -1, 0, -1, 808...","[0.0, -1.0, 59.0, 0.0, -1.0, 80.0, 5.0, 0.0, 0...",[0]


In [22]:
# create a experimentation decklist with same format and push it to builder to build out a dataset. Put sequences into the sequence to text converters to see relation table with names.
# See how accurate model is at predicting card relations withing a deck -> It should predict most cards as 1.

# Create a function which allows you to type 5 cards and the create a dataset with every other card in the game as a subject and see what cards the model predicts will go well with your chosen cards.

Experimentation_Dataset = Dataset_Builder('Dataset/Experimental_Deck_Lists.txt' , True)
pd.DataFrame(Experimentation_Dataset)    

['Nibiru, the Primal Being', 'D.D. Crow', 'Dark Ruler No More', 'D.D. Crow', 'D.D. Crow']


,0,1,2
0,"[276, -1, 363, 0, -1, 85, 8, 0, 0, -1, 138, -1...","[465.0, -1.0, 414.0, 0.0, -1.0, 21.0, 8.0, 0.0...",[0]
1,"[0, -1, 59, 0, -1, 111, 5, 0, 0, -1, 0, -1, 28...","[61.0, -1.0, 387.0, 0.0, -1.0, 21.0, 8.0, 0.0,...",[0]
2,"[0, -1, 59, 0, -1, 80, 5, 0, 0, -1, 0, -1, 547...","[0.0, -1.0, 59.0, 0.0, -1.0, 80.0, 5.0, 0.0, 0...",[1]
3,"[0, -1, 59, 0, -1, 111, 5, 0, 0, -1, 0, -1, 0,...","[0.0, -1.0, 387.0, 0.0, -1.0, 120.0, 8.0, 0.0,...",[1]
4,"[465, -1, 371, 0, -1, 93, 8, 0, 0, -1, 138, -1...","[97.0, -1.0, 371.0, 0.0, -1.0, 21.0, 8.0, 0.0,...",[1]
...,...,...,...
499,"[114, -1, 371, 0, -1, 21, 8, 0, 0, -1, 138, -1...","[334.0, -1.0, 371.0, 0.0, -1.0, 93.0, 8.0, 0.0...",[1]
500,"[114, -1, 374, 0, -1, 93, 8, 0, 0, -1, 198, -1...","[61.0, -1.0, 411.0, 0.0, -1.0, 75.0, 8.0, 0.0,...",[1]
501,"[114, -1, 301, 0, -1, 21, 8, 0, 0, -1, 138, -1...","[0.0, -1.0, 327.0, 0.0, -1.0, 111.0, 5.0, 0.0,...",[1]
502,"[114, -1, 371, 0, -1, 21, 8, 0, 0, -1, 138, -1...","[0.0, -1.0, 59.0, 0.0, -1.0, 111.0, 5.0, 0.0, ...",[1]


In [213]:
Tokenized_sequence_database
   

array([[  0,  -1, 327, ...,   0,   0,   0],
       [  0,  -1, 327, ...,   0,   0,   0],
       [  0,  -1, 711, ...,   0,   0,   0],
       ...,
       [194,  -1,  99, ...,   0,   0,   0],
       [114,  -1, 866, ...,   0,   0,   0],
       [114,  -1, 146, ...,   0,   0,   0]])

In [28]:
def positional_encoding(length, depth):
  depth = depth/2
  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate( [np.sin(angle_rads), np.cos(angle_rads)], axis=-1) 

  return pos_encoding

class Positional_Embedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model ):
    super().__init__()

    self.d_model = d_model
    
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=False) 
    self.pos_encoding = positional_encoding(length=905, depth=d_model)
    
  def call(self, x):
    x = self.embedding(x)
    
    #x*= np.sqrt(self.d_model) # Scale Values by their embedding dimensionality otherwise they could get overwhelmed by positional encoder
    #x = x + self.pos_encoding 
    return x





class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

class DeciderSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, ffn, dropout_rate):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(ffn, activation='relu'), 
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x) 
    return x
  
class EncoderLayer(tf.keras.layers.Layer):
  """Single Encoder Layer with DeciderMHA and Feed Forward layer"""
  def __init__(self, d_model, ffn , dropout_rate ):
    super().__init__()

    self.DSA = DeciderSelfAttention(num_heads=4, key_dim=100)       # Scaling Number of Heads increases parameters as this is a different implementation of mha compared to attention is all you need paper.
    self.FF = FeedForward(d_model, ffn , dropout_rate)

  def call(self, x):
    
    
    x = self.DSA(x)

    x = self.FF(x)

    return x

class Encoder(tf.keras.layers.Layer):
  """Full Encoder with embedding layer with dropout and encoder layers"""
  def __init__(self, d_model, vocab_size , ffn , dropout_rate , num_layers):
    super().__init__()

    self.num_layers = num_layers

    self.Pos_Embedding = Positional_Embedding(vocab_size, d_model)
    self.EL = [EncoderLayer(d_model, ffn , dropout_rate) for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self,x):
    
    x = self.Pos_Embedding(x)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.EL[i](x)

    return x



class SubjectSelfAttention(BaseAttention):
  
  def call(self, x):
    
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

class SubjectCrossAttention(BaseAttention):

  def call(self, x , context):
    attn_output = self.mha(
        query=x,
        value=context,
        key=context)
    
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

class ComparatorLayer(tf.keras.layers.Layer):

  def __init__(self , d_model, ffn , dropout_rate):
    super().__init__()

    self.SSA = SubjectSelfAttention(num_heads=4, key_dim=100)      
    self.SCA = SubjectCrossAttention(num_heads=4, key_dim=100)
    self.FF = FeedForward(d_model, ffn , dropout_rate)

  def call(self, x , context):
    
    x = self.SSA(x)

    x = self.SCA(x , context)

    x = self.FF(x)

    return x
  
class Comparator(tf.keras.layers.Layer):
  
  def __init__(self, d_model, vocab_size , ffn , dropout_rate , num_layers):
    super().__init__()

    self.num_layers = num_layers

    self.Pos_Embedding = Positional_Embedding(vocab_size, d_model)
    self.CL = [ComparatorLayer(d_model, ffn , dropout_rate) for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x , context):
    
    x = self.Pos_Embedding(x)
    
    x = self.dropout(x)
    
    for i in range(self.num_layers):
      x = self.CL[i](x , context)

    return x

class FinalFeedForward(tf.keras.layers.Layer):
  def __init__(self , dropout_rate):
    super().__init__()

    self.seq = tf.keras.Sequential([
      tf.keras.layers.Flatten(),          #Flattens sentances for each card comparision , into a single 1d array , so it can generate probabilities properly, instead of shoving 100 x905 matrix straight through and generating 100 probabilities for each card comparision feature embedding
      tf.keras.layers.Dense(50, activation='relu'),
      tf.keras.layers.Dense(25, activation='relu'),
      tf.keras.layers.Dropout(dropout_rate),
      tf.keras.layers.Dense(1 , activation='sigmoid')
      
    ])
    
  def call(self, x):
    
    x = self.seq(x) 
    return x
  
class FullModel(tf.keras.Model):
   def __init__(self, d_model, vocab_size , ffn , dropout_rate , num_layers):
    super().__init__()

    self.enc = Encoder(d_model, vocab_size , ffn , dropout_rate , num_layers)
    self.com = Comparator(d_model, vocab_size , ffn , dropout_rate , num_layers)
    self.FFF = FinalFeedForward(dropout_rate)

   def call(self, inputs):
     context , x = inputs
     
     
     
     context = self.enc(context)
    
     x = self.com(x , context)

     x = self.FFF(x)

     return x
   
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [221]:
subject_card

<tf.Tensor: shape=(1, 905), dtype=int32, numpy=
array([[ 97,  -1, 146,   0,  -1,  59,   8,   0,   0,  -1, 279,  -1,   0,
          0,   0,   0,   0,   0,   0,   0,   0,  -1, 150,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
   

In [13]:

#subject_card = Positional_Embedding(12647 , 100 )(subject_card)

emb = Positional_Embedding(12647 , 100)
comp_test = Comparator(100 , 12647 , 1000 , 0.3 , 2)
compL_test = ComparatorLayer(100 , 1000 , 0.3)
FM_Test = FullModel(100 , 12647 , 1000 , 0.3 , 2)

In [279]:
emb(subject_card)

<tf.Tensor: shape=(1, 905, 100), dtype=float32, numpy=
array([[[-0.04667787, -0.00644787,  0.00574112, ...,  0.01819483,
          0.02517973, -0.01179622],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.00734416,  0.04132183, -0.01791579, ..., -0.03784325,
          0.03174325, -0.04850903],
        ...,
        [-0.04667787, -0.00644787,  0.00574112, ...,  0.01819483,
          0.02517973, -0.01179622],
        [-0.04667787, -0.00644787,  0.00574112, ...,  0.01819483,
          0.02517973, -0.01179622],
        [-0.04667787, -0.00644787,  0.00574112, ...,  0.01819483,
          0.02517973, -0.01179622]]], dtype=float32)>

In [19]:
    
input_array = ['Galaxy Wizard' , 'Galaxy Soldier' , 'Photon Orbital' , 'Galaxy-Eyes Photon Dragon' ,'Galaxy Summoner']

decider1 = tf.convert_to_tensor([stitcher(0 , [] , False, input_array)])
subject_card1 = tf.convert_to_tensor(    [np.concatenate( (Tokenized_sequence_database[7] , np.zeros(724)) , axis=None )]     ) 


# decider2 = tf.convert_to_tensor([stitcher(0 , [] , False, input_array)])
# subject_card2 = tf.convert_to_tensor(    [np.concatenate( (Tokenized_sequence_database[8434] , np.zeros(724)) , axis=None )]     ) 
# sm = 0
# for i in (compL_test(emb(subject_card1) , emb(decider1)) - compL_test(emb(subject_card2) , emb(decider2)))[0][0]:
#     sm+=i
#compL_test(emb(subject_card1) , emb(decider1))
# print(sm) 
#print(Model.predict((decider , subject_card))[0])
#sum(abs((compL_test(emb(subject_card1) , emb(decider1)) - compL_test(emb(subject_card2) , emb(decider2)))[0][0]))
FM_Test((decider1 , subject_card1))          

Enc ran
In comparator, before passing through pos embedding x , has a shape of:
tf.Tensor([  1 905], shape=(2,), dtype=int32)
In comparator, after passing through pos embedding x , has a shape of:
tf.Tensor([  1 905 100], shape=(3,), dtype=int32)
In comparator, after passing through dropout x , has a shape of:
tf.Tensor([  1 905 100], shape=(3,), dtype=int32)
In ComparatorLayer, before SSa , x has a shape of:
tf.Tensor([  1 905 100], shape=(3,), dtype=int32)
In SSa , x has a shape of:
tf.Tensor([  1 905 100], shape=(3,), dtype=int32)
In ComparatorLayer, before SSa , x has a shape of:
tf.Tensor([  1 905 100], shape=(3,), dtype=int32)
In SSa , x has a shape of:
tf.Tensor([  1 905 100], shape=(3,), dtype=int32)


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.42610976]], dtype=float32)>

In [31]:

def Train_Gen():
    seti0 = [i[0] for i in Training_Validation_Dataset]
    seti1 = [i[1] for i in Training_Validation_Dataset]
    seti2 = [i[2] for i in Training_Validation_Dataset]
    for _ in range(2000):
        r_int = random.randint(0 , len(Training_Validation_Dataset) - 20)
        train_seti0 = tf.convert_to_tensor(seti0[r_int:r_int+20])
        train_seti1 = tf.convert_to_tensor(seti1[r_int:r_int+20])
        train_seti2 = tf.convert_to_tensor(seti2[r_int:r_int+20])
    
        yield (train_seti0,train_seti1), train_seti2

def Val_Gen():
    seti0 = [i[0] for i in Experimentation_Dataset]
    seti1 = [i[1] for i in Experimentation_Dataset]
    seti2 = [i[2] for i in Experimentation_Dataset]
    for _ in range(1000):
        r_int = random.randint(0 , len(Experimentation_Dataset) - 20)
        val_seti0 = tf.convert_to_tensor(seti0[r_int:r_int+20])
        val_seti1 = tf.convert_to_tensor(seti1[r_int:r_int+20])
        val_seti2 = tf.convert_to_tensor(seti2[r_int:r_int+20])

        yield (val_seti0,val_seti1), val_seti2

def Experimentation_Gen():
    seti0 = [i[0] for i in Experimentation_Dataset]
    seti1 = [i[1] for i in Experimentation_Dataset]
    seti2 = [i[2] for i in Experimentation_Dataset]

    for _ in range(len(Experimentation_Dataset)):
        # For some reason i doesnt iterate and gets stuck at 0 , so had to use random int
        r_int = random.randint(0 , 90)
        exp_seti0 = tf.convert_to_tensor(seti0[r_int:r_int+1])
        exp_seti1 = tf.convert_to_tensor(seti1[r_int:r_int+1])
        exp_seti2 = tf.convert_to_tensor(seti2[r_int:r_int+1])

        yield (exp_seti0,exp_seti1), exp_seti2



In [32]:
#Testing accuracy is much higher than training accuracy. This is due to dropouts causing lower accuracy during training but giving a more robust model when testing
Model = FullModel(100 , 12647 , 1000 , 0.2 , 2)

learning_rate = CustomSchedule(d_model = 100)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

Model.compile(
    loss= tf.keras.losses.BinaryCrossentropy(),
    optimizer=optimizer,
    metrics= 'accuracy' )

history = Model.fit(Train_Gen() , epochs=20, 
                               validation_data = Val_Gen()  , steps_per_epoch=10 , batch_size=20 , validation_steps=10 , validation_batch_size=20)

Epoch 1/20
10/10 [==============================] - 14s 1s/step - loss: 0.7278 - accuracy: 0.5050 - val_loss: 0.7207 - val_accuracy: 0.5260
Epoch 2/20
10/10 [==============================] - ETA: 0s - loss: 0.7305 - accuracy: 0.5500

ResourceExhaustedError: Graph execution error:

Detected at node 'full_model_4/comparator_6/comparator_layer_14/subject_self_attention_14/multi_head_attention_38/softmax_26/Softmax' defined at (most recent call last):
    File "c:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 978, in launch_instance
      app.start()
    File "c:\Users\X99S5\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "

In [384]:
def predict_matches():
    #input_array = [input('enter') for _ in range(5)]
    input_array = ['Galaxy Wizard' , 'Galaxy Soldier' , 'Photon Orbital' , 'Galaxy-Eyes Photon Dragon' ,'Galaxy Summoner']
    
    decider = tf.convert_to_tensor([stitcher(0 , [] , False, input_array)])
    count = 0
    for indx in pd.DataFrame(Tokenized_sequence_database).index.values:
        if count<2500:
            pass
        else:
            if (count % 100) == 0:
                print(count)
            subject_card = tf.convert_to_tensor([np.concatenate( (Tokenized_sequence_database[indx] , np.zeros(724)) , axis=None )])
            #subject_card = tf.convert_to_tensor([np.zeros(905) ])
            #print(subject_card)
            print(Model.predict((decider , subject_card))[0])
            time.sleep(1)
            
            # if Model.predict((decider , subject_card) , verbose=False)[0][0] > 0.1:
            #     print(Sliced_df[indx:indx+1]['name'])
            #     break
        count+=1
predict_matches()


2500
Enc ran
Com ran
1/1 [==============================] - 1s 837ms/step
[0.6818564]
1/1 [==============================] - 0s 30ms/step
[0.6818564]
1/1 [==============================] - 0s 25ms/step
[0.6818564]
1/1 [==============================] - 0s 38ms/step
[0.6818564]
1/1 [==============================] - 0s 111ms/step
[0.6818564]
1/1 [==============================] - 0s 64ms/step
[0.6818564]
1/1 [==============================] - 0s 139ms/step
[0.6818564]
1/1 [==============================] - 0s 118ms/step
[0.6818564]
1/1 [==============================] - 0s 55ms/step
[0.6818564]
1/1 [==============================] - 0s 246ms/step
[0.6818564]
1/1 [==============================] - 0s 26ms/step
[0.6818564]
1/1 [==============================] - 0s 54ms/step
[0.6818564]
1/1 [==============================] - 0s 50ms/step
[0.6818564]
1/1 [==============================] - 0s 258ms/step
[0.6818564]
1/1 [==============================] - 0s 25ms/step
[0.6818564]
1/1 [========

KeyboardInterrupt: 

In [141]:
pred = pd.DataFrame(Model.predict(Experimentation_Gen()))
count = 0
for i in pred[0]:
    if i > 0.5:
        count+=1
print(count/len(pred))


504/504 [==============================] - 8s 12ms/step
0.16468253968253968


In [220]:
k , trash = next(Experimentation_Gen())
l1 , l2 = k
singular_pred = Model.predict(k)
singular_pred

1/1 [==============================] - 0s 219ms/step


array([[0.9966325]], dtype=float32)

In [222]:

tokenizer.sequences_to_texts([l2[0].numpy()])

["2 aqua effect monster water cannot be used as a synchro material. this card's name becomes des frog while it is on the field. if this card is in your graveyard: you can banish 1 frog monster from your graveyard; special summon this card."]

In [134]:
200 % 100

0